In [25]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

In [26]:
df = pd.read_csv("../processed-data.csv")
df

,Date,Value,MovingAvg,PercAvg_Pred,ExpSmooth,RMA,RMA_Pred
0,2010-01-01,388.91,NaN,407.037443,388.91000,NaN,NaN
1,2010-02-01,390.41,NaN,407.902507,389.36000,NaN,NaN
2,2010-03-01,391.37,NaN,408.657562,389.96300,NaN,NaN
3,2010-04-01,392.67,NaN,410.147672,390.77510,NaN,NaN
4,2010-05-01,393.21,NaN,409.564129,391.50557,NaN,NaN
...,...,...,...,...,...,...,...
203,2026-12-01,NaN,NaN,NaN,NaN,NaN,NaN
204,2027-01-01,NaN,NaN,NaN,NaN,NaN,NaN
205,2027-02-01,NaN,NaN,NaN,NaN,NaN,NaN
206,2027-03-01,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
def evaluateModel(df, predictedColumnName):
    validDf = df.dropna(subset=["Value", predictedColumnName])
    
    if len(validDf) == 0:
        return {
            "mse": np.nan,
            "rmse": np.nan,
            "mae": np.nan,
            "mape": np.nan
        }
    
    actual = validDf["Value"]
    predicted = validDf[predictedColumnName]
    
    mse = np.mean((actual - predicted) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(actual - predicted))
    
    nonZeroActual = actual[actual != 0]
    nonZeroPredicted = predicted[actual != 0]
    
    mape = np.mean(np.abs((nonZeroActual - nonZeroPredicted) / nonZeroActual)) * 100 if len(nonZeroActual) > 0 else np.nan
    
    return {
        "mse": mse,
        "rmse": rmse,
        "mae": mae,
        "mape": mape
    }

In [ ]:
def exponentialSmoothing(data, alpha):
    result = [data[0]] 
    for n in range(1, len(data)):
        result.append(alpha * data[n] + (1 - alpha) * result[n-1])
    return result

def predictWithExponentialSmoothing(data, alpha, forecast_horizon):
    validData = [x for x in data if pd.notna(x)]
    if len(validData) == 0:
        return [np.nan] * forecast_horizon
    
    smoothed = exponentialSmoothing(validData, alpha)
    lastSmoothedValue = smoothed[-1]
    
    recentValues = smoothed[-3:] if len(smoothed) >= 3 else smoothed
    if len(recentValues) >= 2:
        trend = (recentValues[-1] - recentValues[0]) / len(recentValues)
    else:
        trend = 0
    
    predictions = []
    currentValue = lastSmoothedValue
    
    for i in range(forecast_horizon):
        dampingFactor = 0.9 ** i  # exponential dampening
        currentValue = lastSmoothedValue + (trend * (i + 1) * dampingFactor)
        predictions.append(currentValue)
    
    return predictions


In [ ]:
def execExponentialSmoothing(alpha, forecast_horizon):
    dfCopy = df.copy()
    
    actualValues = dfCopy['Value'].dropna().values
    
    smoothedValues = exponentialSmoothing(actualValues, alpha)
    
    dfCopy['ExpSmooth'] = np.nan
    
    actualIndices = dfCopy['Value'].dropna().index
    for i, idx in enumerate(actualIndices):
        dfCopy.loc[idx, 'ExpSmooth'] = smoothedValues[i]
    
    predictions = predictWithExponentialSmoothing(actualValues, alpha, forecast_horizon)
    
    futureRows = dfCopy[dfCopy['Value'].isna() & dfCopy['MovingAvg'].notna()].index
    
    for i, idx in enumerate(futureRows[:len(predictions)]):
        dfCopy.loc[idx, 'ExpSmooth'] = predictions[i]
    
    print(evaluateModel(dfCopy, "ExpSmooth"), "\n")
    
    return dfCopy

In [30]:
df = execExponentialSmoothing(alpha=0.3, forecast_horizon=12)
df

{'mse': np.float64(2.6153383702304565), 'rmse': np.float64(1.6172007822872385), 'mae': np.float64(1.4726006242587604), 'mape': np.float64(0.3616219155351945)} 



,Date,Value,MovingAvg,PercAvg_Pred,ExpSmooth,RMA,RMA_Pred
0,2010-01-01,388.91,NaN,407.037443,388.91000,NaN,NaN
1,2010-02-01,390.41,NaN,407.902507,389.36000,NaN,NaN
2,2010-03-01,391.37,NaN,408.657562,389.96300,NaN,NaN
3,2010-04-01,392.67,NaN,410.147672,390.77510,NaN,NaN
4,2010-05-01,393.21,NaN,409.564129,391.50557,NaN,NaN
...,...,...,...,...,...,...,...
203,2026-12-01,NaN,NaN,NaN,NaN,NaN,NaN
204,2027-01-01,NaN,NaN,NaN,NaN,NaN,NaN
205,2027-02-01,NaN,NaN,NaN,NaN,NaN,NaN
206,2027-03-01,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Export it
outPath = "../processed-data.csv"
df.to_csv(outPath, index=False)
df

,Date,Value,MovingAvg,PercAvg_Pred,ExpSmooth,RMA,RMA_Pred
0,2010-01-01,388.91,NaN,407.037443,388.91000,NaN,NaN
1,2010-02-01,390.41,NaN,407.902507,389.36000,NaN,NaN
2,2010-03-01,391.37,NaN,408.657562,389.96300,NaN,NaN
3,2010-04-01,392.67,NaN,410.147672,390.77510,NaN,NaN
4,2010-05-01,393.21,NaN,409.564129,391.50557,NaN,NaN
...,...,...,...,...,...,...,...
203,2026-12-01,NaN,NaN,NaN,NaN,NaN,NaN
204,2027-01-01,NaN,NaN,NaN,NaN,NaN,NaN
205,2027-02-01,NaN,NaN,NaN,NaN,NaN,NaN
206,2027-03-01,NaN,NaN,NaN,NaN,NaN,NaN
